<a href="https://colab.research.google.com/github/Phissie/API-BackEnd/blob/master/Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rpi-techfundamentals/fall2018-materials/blob/master/10-deep-learning/04-pytorch-mnist.ipynb)


In [3]:
!pip install torch torchvision

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
num_epochs = 30

# CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
train_size = int(0.8 * len(dataset))
validate_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - validate_size
train_dataset, validate_dataset, test_dataset = random_split(dataset, [train_size, validate_size, test_size])

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size_train, shuffle=True)
validate_loader = DataLoader(dataset=validate_dataset, batch_size=batch_size_test, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size_test, shuffle=False)

Files already downloaded and verified


In [25]:
class CnnNet(nn.Module):
    def __init__(self):
        super(CnnNet, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):

        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)


        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)

        x = self.conv3(x)
        x = F.relu (x)
        x = self.pool(x)
        # print(x.shape)

        x = x.view(-1, 128 * 4 * 4)
        x = self.fc1(x)
        x = F.relu(x)

        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)

        return x

In [ ]:
model = CnnNet()
# model = BpNet()

optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.5)

best_model = None
best_accuracy = 0

def train(epoch):
    model.train()
    total_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        if (batch_idx + 1) % 100 == 0:
            print(f'Epoch {epoch}, Batch {batch_idx + 1}: Current Loss = {total_loss / (batch_idx + 1):.4f}')

    average_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch} Training Loss: {average_loss:.4f}')


def validate(epoch):
    global best_model, best_accuracy
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in validate_loader:
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            total += target.size(0)

    accuracy = 100. * correct / total
    print(f'Epoch {epoch} Validation Accuracy: {accuracy:.2f}%')

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model.state_dict()
        print("Best model saved with accuracy: {:.2f}%".format(best_accuracy))

def test():
    model.load_state_dict(best_model)
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'Test Accuracy: {accuracy:.2f}%')

for epoch in range(1, num_epochs + 1):
    train(epoch)
    validate(epoch)
    print('\n')

test()

Epoch 1, Batch 100: Current Loss = 2.2967
Epoch 1, Batch 200: Current Loss = 2.2807
Epoch 1, Batch 300: Current Loss = 2.2376
Epoch 1, Batch 400: Current Loss = 2.1881
Epoch 1, Batch 500: Current Loss = 2.1518
Epoch 1, Batch 600: Current Loss = 2.1184
Epoch 1 Training Loss: 2.1117
Epoch 1 Validation Accuracy: 29.28%
Best model saved with accuracy: 29.28%


Epoch 2, Batch 100: Current Loss = 1.9009
Epoch 2, Batch 200: Current Loss = 1.8626
Epoch 2, Batch 300: Current Loss = 1.8318
Epoch 2, Batch 400: Current Loss = 1.8061
Epoch 2, Batch 500: Current Loss = 1.7790
Epoch 2, Batch 600: Current Loss = 1.7496
Epoch 2 Training Loss: 1.7453
Epoch 2 Validation Accuracy: 41.00%
Best model saved with accuracy: 41.00%


Epoch 3, Batch 100: Current Loss = 1.5875
Epoch 3, Batch 200: Current Loss = 1.5601
Epoch 3, Batch 300: Current Loss = 1.5439
Epoch 3, Batch 400: Current Loss = 1.5330
Epoch 3, Batch 500: Current Loss = 1.5185
Epoch 3, Batch 600: Current Loss = 1.5043
Epoch 3 Training Loss: 1.5013


In [ ]:
class BpNet(nn.Module):
    def __init__(self):
        super(BpNet, self).__init__()

        self.fc1 = nn.Linear(3 * 32 * 32, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 3 * 32 * 32)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

In [ ]:
class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.residual_block1 = self.make_residual_block(64, 64)
        self.residual_block2 = self.make_residual_block(64, 128)
        self.residual_block3 = self.make_residual_block(128, 256)

        self.fc = nn.Linear(256 * 4 * 4, 10)

    def make_residual_block(self, in_channels, out_channels, stride=1):
        layers = []
        if stride != 1 or in_channels != out_channels:
            layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False))
            layers.append(nn.BatchNorm2d(out_channels))
        layers.extend([
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels)
        ])
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))

        x = self.residual_block1(x)
        x = self.residual_block2(x)
        x = self.residual_block3(x)

        x = F.avg_pool2d(x, 4)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)
